In [1]:
# !pip install -r ./requirements.txt -q
# !pip - installs in base environment
# !\pip - installs in virtual environment

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.341
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/chidi/Library/Python/3.9/lib/python/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
# !pip install langchain --upgrade -q

#### Python-dotenv

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
# load_dotenv('./.env')
load_dotenv(find_dotenv(), override=True)

True

In [5]:
os.environ.get('PINECONE_API_KEY')

'091b67e9-39dd-4c7e-aff2-38668b522b6c'

#### LLM Models (Wrappers): GPT-3

In [6]:
from langchain.llms import OpenAI

In [7]:
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [8]:
output = llm('explain the meaning of "bottom barrel" in one sentence')
print(output)



Bottom barrel refers to the lowest quality or worst of a given range of items.


In [9]:
print(llm.get_num_tokens('explain the meaning of "bottom barrel" in one sentence'))

12


In [10]:
output = llm.generate(['Antanarivo is the capital of ...', 
                   'What is the formula for the perimeter of a sphere?'])

In [11]:
print(output.generations)

[[Generation(text='\n\nMadagascar.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the perimeter of a sphere is 2πr, where r is the radius of the sphere.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [14]:
print(output.generations[0][0].text)



Madagascar.


In [15]:
len(output.generations)

2

In [16]:
output = llm.generate(['Write an original tagline for a burger restaurant']*3)

In [18]:
for o in output.generations:
    print(o[0].text)



"Burger Bliss - Satisfy Your Cravings!"


"Sink your teeth into the juiciest burgers around!"


"Come for the Burgers, Stay for the Fries!"


#### ChatModels: GPT-3.5-Turbo and GPT-4

In [19]:
from langchain.schema import(
    AIMessage, # Assistant message of OpenAI API
    HumanMessage, # User message of OpenAI API
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [20]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a polyglot and respond only in Spanish.'),
    HumanMessage(content='What is "My Heart" in your language?')
]
output = chat(messages)

In [21]:
print(output.content)

"Mi corazón" is the translation of "My Heart" in Spanish.


#### Prompt Templates

In [22]:
from langchain import PromptTemplate

In [23]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['language', 'virus'] template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.'


In [25]:
from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='ebola', language='spanish'))
print(output)



El ébola es un virus mortal que se transmite a través de fluidos corporales. Los síntomas incluyen fiebre, dolor de cabeza, fatiga y vómitos. Puede ser fatal si no se trata adecuadamente. Es importante que las personas tomen precauciones para evitar la propagación del virus.


#### Simple Chains

In [27]:
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
# output = chain('Staph')
output = chain.run({'virus':'HSV', 'language':'french'})


In [28]:
print(output)

Le HSV (Herpes Simplex Virus) est un virus de la famille des herpèsviridés qui infecte les êtres humains. Il existe deux types de HSV : le HSV-1 et le HSV-2. Le HSV-1 est principalement responsable des infections buccales, telles que les boutons de fièvre, tandis que le HSV-2 est généralement associé aux infections génitales. Ces virus sont très contagieux et se transmettent par contact direct avec les lésions cutanées ou les muqueuses infectées. Bien qu'ils ne puissent pas être guéris, les symptômes peuvent être traités avec des médicaments antiviraux pour réduire la fréquence et la gravité des poussées.


#### Sequential Chains

In [30]:
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')



> Entering new SimpleSequentialChain chain...
.

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
The Python function softmax() computes the softmax values for each set of scores in x. It takes in an array or matrix of scores as input. 

The function uses the numpy library to perform mathematical operations. 

The softmax calculation is achieved by exponentiating each element in the input array (using np.exp(x)) and then dividing the resulting array by the sum of all exponentiated values along the specified axis (using np.sum(np.exp(x), axis=0)). 

The axis parameter determines the axis along which the sum is performed. In this case, axis=0 indicates that the sum is performed vertically, for each column of the input matrix. 

Finally, the function returns the softmax values, which are the resulting array obtained from dividing each exponentiated value by the sum.

> Finished chain.


#### LangChain Agents

In [31]:
5.1**7.3

146306.05007233328

In [32]:
# !pip install langchain_experimental -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [33]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

In [34]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm, 
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.run('Calculate the square root of the factorial of 20 \
                   and display it with 4 decimal points')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to calculate the factorial of 20 and then take the square root
Action: Python_REPL
Action Input: from math import factorial
               print(round(math.sqrt(factorial(20)), 4))
Observation: IndentationError('unexpected indent', ('<string>', 2, 15, '               print(round(math.sqrt(factorial(20)), 4))\n'))
Thought: I need to remove the extra indentation
Action: Python_REPL
Action Input: from math import factorial
print(round(math.sqrt(factorial(20)), 4))
Observation: NameError("name 'math' is not defined")
Thought: I need to import the math module
Action: Python_REPL
Action Input: import math
from math import factorial
print(round(math.sqrt(factorial(20)), 4))
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'

In [36]:
from math import factorial

round(factorial(20)**0.5,4)

1559776268.6285

In [37]:
agent_executor.run('what is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
 I need to use the Python REPL to calculate this
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'